In [1]:
%load_ext autoreload
%autoreload 2

import os

from coeditor.c3problem import C3ProblemGenerator, C3ProblemTokenizer
from coeditor.common import *
from coeditor.dataset import make_or_load_dataset
from coeditor.model import (
    DecodingArgs,
    MultiRoundEvaluator,
    MultiRoundStrategy,
    RetrievalEditorModel,
)

os.chdir(proj_root())

In [2]:
dataset_name = "perm2k"
model_name = "MrVPlusOne/coeditor-perm2k-base-v1.7.3"
model_device = "cuda"
N_test = 1000

testset = make_or_load_dataset(
    dataset_name,
    C3ProblemGenerator(),
    splits=("test",),
    time_limit_per_commit=40,
)["test"]

print(f"{len(testset)}")
subset = random_subset(testset, N_test, rng=42)
print(f"{len(subset)=}")

pmap: _process_commits: 100%|██████████| 50/50 [00:05<00:00,  9.31repo/s]


Dataset total size (n=1649): 5150.76 MB
22516
len(subset)=1000


In [3]:
import numpy as np

target_file = (
    proj_root() / f"output/multi_round_eval/{model_name}/most_uncertain-{N_test}.pkl"
)
metric_stats = pickle_load(target_file)

In [4]:
sample_ids = [i for i, m in enumerate(metric_stats) if 50 <= m["keystrokes"].total_edit_gain <= 100 and m["diff-lines"].rounds == 1]
print(f"{len(sample_ids)=}")

len(sample_ids)=33


In [5]:
tokenizer = C3ProblemTokenizer.for_eval()
dec_args = DecodingArgs(do_sample=False, num_beams=1)
model = RetrievalEditorModel.load(model_name)
model.to(model_device)
None

In [6]:
# target_hash="30eb3afbe173b75dc5ec44348cef49fe3eac2421"
# ex_id = [i for i in sample_ids if subset[i].src_info["commit"].hash == target_hash][0]
ex_id = sample_ids[5]
ex = subset[ex_id]
evaluator = MultiRoundEvaluator(model, tokenizer, dec_args, strategy="pick_first")

evaluator.multi_round_edit_gain(ex, print_steps=True, skip_ctx=False)

path: db.basedb/BaseDB._insert
n_references: 5
total_reference_tks: 1125
project: qiandao-today~qiandao
commit: [8d0f0e04df Cursor操作结束后自动关闭; 统一DB连接操作]
{'input_tks': 234, 'output_tks': 44, 'n_references': 5, 'changed_reference_tks': 957, 'unchanged_reference_tks': 168, 'total_reference_tks': 1125}
--------------------------------------------------------------------------------
round:
1
========Ground Truth========
<13>:<add>         lastrowid = dbcur.lastrowid
     <add>         dbcur.close()
     <add>         return lastrowid
     <del>         return dbcur.lastrowid

========Main Code========
      # module: db.basedb
          
      
      class BaseDB(object):
       
          def _insert(self, tablename=None, **values):
 <0>          tablename = self.escape(tablename or self.__tablename__)
 <1>          if values:
 <2>              _keys = ", ".join((self.escape(k) for k in values.keys()))
 <3>              _values = ", ".join([self.placeholder, ] * len(values))
 <4>            

{'keystrokes': MultiRoundEditStats(label_edit_gain=69, first_edit_gain=69, total_edit_gain=69, rounds=1),
 'diff-lines': MultiRoundEditStats(label_edit_gain=4, first_edit_gain=4, total_edit_gain=4, rounds=1),
 'levenshtein': MultiRoundEditStats(label_edit_gain=55, first_edit_gain=55, total_edit_gain=55, rounds=1)}